In [1]:
# ==============================================================================
# SCRIPT TO GENERATE FINAL PERFORMANCE REPORT (CACHE FIX)
# This version corrects the CombinedDataset to use in-memory caching,
# resolving the extreme performance bottleneck.
# ==============================================================================

import os
import sys
from google.colab import auth, drive

# --- 1. SETUP THE COLAB ENVIRONMENT ---
print("--- Step 1: Preparing the main Colab environment ---")
try:
    auth.authenticate_user()
    print("✅ Authentication successful.")
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    sys.exit(f"❌ FATAL ERROR: Could not set up environment. Details: {e}")

# --- CONFIGURATION (UPDATE THIS) ---
CHECKPOINT_PATH = "/content/drive/MyDrive/final_model_checkpoint/best-model-20250908_233138_convnext_base_2000files_Augmented_cwN1-6.5.ckpt"

# --- Pre-flight check to validate the checkpoint path ---
print("\n--- Step 2: Validating checkpoint path ---")
if not os.path.exists(CHECKPOINT_PATH):
    print(f"❌ FATAL ERROR: Checkpoint file not found at the specified path.")
    sys.exit(f"Path not found: {CHECKPOINT_PATH}")
else:
    print(f"✅ Checkpoint file found: {os.path.basename(CHECKPOINT_PATH)}")


# --- 2. CREATE AND PROVISION THE ISOLATED REPORTING ENVIRONMENT ---
print("\n--- Step 3: Creating and provisioning the isolated reporting environment ---")
!pip install --upgrade -q virtualenv
!virtualenv report_env
!report_env/bin/pip install --upgrade -q pip "pytorch-lightning" "timm" "pandas>=2.0" "pyarrow>=15.0" "fsspec>=2023.6.0" gcsfs google-auth matplotlib seaborn scikit-learn
print("✅ All dependencies installed successfully into 'report_env'.")


# --- 3. CREATE AND RUN THE REPORTING LOGIC SCRIPT ---
print("\n--- Step 4: Preparing and executing the reporting logic ---")

python_script_logic = f'''
import matplotlib
matplotlib.use('Agg')
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassConfusionMatrix
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

torch.set_float32_matmul_precision('medium')

# --- CONFIGURATION ---
CHECKPOINT_PATH = "{CHECKPOINT_PATH}"
GCS_MANIFEST_PATH = "gs://shhs-sleepedfx-data-bucket/metadata/shhs_dataset_manifest.csv"
NUM_FILES_TO_USE = 2000
BATCH_SIZE = 256
NUM_WORKERS = 2

# --- DEFINITIONS (Must match the training script) ---
def get_model(model_name='convnext_base', pretrained=False):
    model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k', pretrained=pretrained)
    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
    with torch.no_grad():
        if original_conv.weight.shape[1] == 3:
            new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
    model.stem[0] = new_first_conv
    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, 5)
    return model

class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name='convnext_base', learning_rate=2e-5, class_weights=None, epochs=40):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name)
        self.loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float) if class_weights else None)
    def forward(self, x): return self.model(x)
    def normalize_on_gpu(self, x):
        mean = torch.mean(x, dim=(1, 2, 3), keepdim=True)
        std = torch.std(x, dim=(1, 2, 3), keepdim=True)
        return (x - mean) / (std + 1e-6)

class CombinedDataset(Dataset):
    # --- MODIFICATION: Restored the essential in-memory cache ---
    def __init__(self, manifest_path, num_files=None):
        manifest_df = pd.read_csv(manifest_path)
        if num_files: manifest_df = manifest_df.head(num_files)
        self.file_paths = manifest_df['file_path'].tolist()
        self.cumulative_epochs = np.cumsum(manifest_df['epoch_count'].values)
        self.total_epochs = self.cumulative_epochs[-1]
        self._cache = {{}} # The cache is essential for performance

    def __len__(self): return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]

        # This logic ensures each worker only reads a file from GCS ONCE.
        if file_path not in self._cache:
            df = pd.read_parquet(file_path)
            self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)

        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        spectrogram_2d = spectrogram_flat.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print(f"🧠 Evaluating model from: {os.path.basename(CHECKPOINT_PATH)}")

    print("\\n--- Loading and splitting dataset from GCS ---")
    full_dataset = CombinedDataset(GCS_MANIFEST_PATH, num_files=NUM_FILES_TO_USE)
    torch.manual_seed(42)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    _, val_dataset = random_split(full_dataset, [train_size, val_size])
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    print("✅ Validation data is ready.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_dir = os.path.dirname(CHECKPOINT_PATH)
    exp_name = Path(CHECKPOINT_PATH).stem.replace('best-model-', '')

    model = SleepStageClassifierLightning.load_from_checkpoint(checkpoint_path=CHECKPOINT_PATH)
    model.to(device)
    model.eval()

    print("\\n--- Generating Final Performance Metrics (this may take a few minutes)... ---")
    all_preds, all_labels = [], []
    with torch.no_grad():
        for i, (x, y) in enumerate(val_loader):
            print(f"\\r  -> Predicting on batch {{i+1}}/{{len(val_loader)}}", end="")
            x_normalized = model.normalize_on_gpu(x.to(device))
            logits = model(x_normalized)
            all_preds.append(torch.argmax(logits, dim=1).cpu())
            all_labels.append(y.cpu())
    print("\\n  -> Prediction complete.")

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    stage_map = {{0: "Wake", 1: "N1", 2: "N2", 3: "N3", 4: "REM"}}
    target_names = [stage_map[i] for i in range(5)]

    print("\\n" + "="*80)
    print("--- Detailed Classification Report ---")
    report = classification_report(all_preds, all_labels, target_names=target_names, digits=4)
    print(report)
    print("="*80)

    print("\\n--- Generating Confusion Matrix Heatmap ---")
    matrix = MulticlassConfusionMatrix(num_classes=5)(torch.tensor(all_preds), torch.tensor(all_labels)).numpy()
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.title(f'Confusion Matrix\\n({{exp_name}})', fontsize=16)
    plt.ylabel('True Label'); plt.xlabel('Predicted Label')
    save_path = os.path.join(save_dir, f"{{exp_name}}_confusion_matrix.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ Confusion matrix plot saved to: {{save_path}}")
'''

# Write the script to a file
with open("run_report.py", "w") as f:
    f.write(python_script_logic)

# Execute the script using the virtual environment's python, forcing the correct backend
!MPLBACKEND=Agg report_env/bin/python run_report.py

print("\n--- Script execution finished. ---")

--- Step 1: Preparing the main Colab environment ---
✅ Authentication successful.
Mounted at /content/drive
✅ Google Drive mounted successfully.

--- Step 2: Validating checkpoint path ---
✅ Checkpoint file found: best-model-20250908_233138_convnext_base_2000files_Augmented_cwN1-6.5.ckpt

--- Step 3: Creating and provisioning the isolated reporting environment ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 32.7 MB/s eta 0:00:00
created virtual environment CPython3.12.11.final.0-64 in 240ms
  creator CPython3Posix(dest=/content/report_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
✅ All dependencies installed successfully into 'report_env'.

--- 